In [9]:
import tensorflow as tf
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re,string,unicodedata
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
print ( tf . version . VERSION )


2.12.0


In [10]:
data = pd.read_csv('IMDB Dataset.csv')
print(data)


                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [11]:
data.describe()


,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [12]:
data['sentiment'].value_counts()


positive    25000
negative    25000
Name: sentiment, dtype: int64

In [13]:
#split the dataset  
#train dataset
train_reviews=data.review[:40000]
train_sentiments=data.sentiment[:40000]
#test dataset
test_reviews=data.review[40000:]
test_sentiments=data.sentiment[40000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(40000,) (40000,)
(10000,) (10000,)


In [14]:
#Tokenization of text
tokenizer=nltk.tokenize.toktok.ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [15]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
data['review']=data['review'].apply(denoise_text)


C:\Users\arthu\AppData\Local\Temp\ipykernel_7048\3553002382.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [24]:
import tensorflow as tf
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re,string,unicodedata
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from keras.preprocessing.text import text_to_word_sequence
from collections import defaultdict
from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt




Exercice 3: 

In [12]:
import os
from collections import defaultdict
from keras.preprocessing.text import text_to_word_sequence, Tokenizer

def get_info(path: str):
    data = list(os.walk(path))[1:]
    files = []
    for d in data:
        folder_name = d[0]
        for file in d[2]:
            files.append((folder_name.split('/')[-1], os.path.join(folder_name, file)))

    d = defaultdict(int)
    texts = defaultdict(list)
    for (cate, file) in files:
        with open(file, 'r') as outfile:
            text = outfile.read()
            texts[cate].append(text)
            words = text_to_word_sequence(text)
            for word in words:
                d[word] += 1
    words = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return (texts, words)


In [13]:
datatest = get_info("C:/Users/arthu/OneDrive/Documents/Cours/ESIR3/Gitlab/AI_LEGRAND_MORVAN/ohsumed-first-20000-docs/test")
datatraining = get_info("C:/Users/arthu/OneDrive/Documents/Cours/ESIR3/Gitlab/AI_LEGRAND_MORVAN/ohsumed-first-20000-docs/training")
english_stops = set(stopwords.words('english'))

In [27]:
import os
from collections import defaultdict
from keras.preprocessing.text import text_to_word_sequence
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

def get_info(path: str):
    data = list(os.walk(path))[1:]
    files = []
    for d in data:
        folder_name = d[0]
        for file in d[2]:
            files.append((folder_name.split('/')[-1], os.path.join(folder_name, file)))

    d = defaultdict(int)
    texts = defaultdict(list)
    for (cate, file) in files:
        with open(file, 'r') as outfile:
            text = outfile.read()
            texts[cate].append(text)
            words = text_to_word_sequence(text)
            for word in words:
                d[word] += 1
    words = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return (texts, words)

# Get data for training and testing sets
datatest = get_info("C:/Users/arthu/OneDrive/Documents/Cours/ESIR3/Gitlab/AI_LEGRAND_MORVAN/ohsumed-first-20000-docs/test")
datatraining = get_info("C:/Users/arthu/OneDrive/Documents/Cours/ESIR3/Gitlab/AI_LEGRAND_MORVAN/ohsumed-first-20000-docs/training")

# Additional imports
from nltk.corpus import stopwords

# Function to remove stopwords and punctuation
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
    return filtered_words

# Explore the data and compute basic statistics
def explore_data(data):
    categories = list(data[0].keys())

    for category in categories:
        print(f"\nCategory: {category}")
        category_texts = data[0][category]

        # Number of examples per category
        num_examples = len(category_texts)
        print(f"Number of Examples: {num_examples}")

        # Vocabulary size per category
        category_words = [preprocess_text(text) for text in category_texts]
        flat_category_words = [word for sublist in category_words for word in sublist]
        category_vocab_size = len(set(flat_category_words))
        print(f"Vocabulary Size: {category_vocab_size}")

        # Word frequency per category
        category_word_freq = FreqDist(flat_category_words)
        common_words = category_word_freq.most_common(10)
        least_common_words = category_word_freq.most_common()[:-10-1:-1]

        print("Common Words:", common_words)
        print("Least Common Words:", least_common_words)

# Explore training and testing data
explore_data(datatraining)
explore_data(datatest)


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\arthu/nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\share\\nltk_data'
    - 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib\\nltk_data'
    - 'C:\\Users\\arthu\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
